In [1]:
import numpy as np

In [2]:
def simulate_game_3players(c1, c2_in, c2_out, c3_in_in, c3_in_out, c3_out_in, num_trials=100_000,seed=2025):
    """
    Simulate the 3-player, 1-toed game given the cutoff thresholds.
    
    Parameters:
        c1 (float): Player 1's cutoff.
        c2_in (float): Player 2's cutoff when Player 1 is in.
        c2_out (float): Player 2's cutoff when Player 1 is out.
        c3_in_in (float): Player 3's cutoff when both P1 and P2 are in.
        c3_in_out (float): Player 3's cutoff when P1 is in and P2 is out.
        c3_out_in (float): Player 3's cutoff when P1 is out and P2 is in.
        num_trials (int): Number of simulation trials.
    
    Returns:
        avg_payoff: tuple of average payoffs (p1, p2, p3).
    """
    # Draw private signals for each player uniformly from [0,1]
    np.random.seed(seed)
    u1 = np.random.rand(num_trials)
    u2 = np.random.rand(num_trials)
    u3 = np.random.rand(num_trials)
    
    # Player 1 decision: "in" if u1 >= c1.
    p1_in = (u1 >= c1)
    
    # Player 2 decision:
    # If Player 1 is in, use cutoff c2_in; if P1 is out, use c2_out.
    p2_in = np.where(p1_in, (u2 >= c2_in), (u2 >= c2_out))
    
    # Player 3 decision:
    # If both P1 and P2 are out, player 3 is automatically in.
    # Otherwise, choose cutoff according to the observed decisions.
    p3_in = np.zeros(num_trials, dtype=bool)
    
    # Case: both P1 and P2 are out -> Player 3 is automatically in.
    auto_in = (~p1_in) & (~p2_in)
    p3_in[auto_in] = True
    
    # When P1 and P2 are in:
    idx = p1_in & p2_in
    p3_in[idx] = (u3[idx] >= c3_in_in)
    
    # When P1 is in and P2 is out:
    idx = p1_in & (~p2_in)
    p3_in[idx] = (u3[idx] >= c3_in_out)
    
    # When P1 is out and P2 is in:
    idx = (~p1_in) & p2_in
    p3_in[idx] = (u3[idx] >= c3_out_in)
    
    # Initialize payoff arrays.
    payoff1 = np.zeros(num_trials)
    payoff2 = np.zeros(num_trials)
    payoff3 = np.zeros(num_trials)
    
    # Count number of players "in" for each trial.
    in_count = p1_in.astype(int) + p2_in.astype(int) + p3_in.astype(int)
    
    # Case 1: Sole player is in -> wins automatically, gets +1.
    sole_p1 = p1_in & (~p2_in) & (~p3_in)
    sole_p2 = (~p1_in) & p2_in & (~p3_in)
    sole_p3 = (~p1_in) & (~p2_in) & p3_in  # When only P3 is in (and automatically in)
    payoff1[sole_p1] = 1
    payoff2[sole_p2] = 1
    payoff3[sole_p3] = 1
    
    # Case 2: Two or more players are in.
    multi_in = in_count >= 2
    if multi_in.any():
        # For players not in, set their modified signal to -infinity.
        u1_mod = np.where(p1_in, u1, -np.inf)
        u2_mod = np.where(p2_in, u2, -np.inf)
        u3_mod = np.where(p3_in, u3, -np.inf)
        # Determine the maximum signal for each trial.
        max_signal = np.maximum(np.maximum(u1_mod, u2_mod), u3_mod)
        
        # Identify winners.
        # Ties are broken in favor of the lower-index player.
        p1_win = multi_in & (u1_mod == max_signal)
        p2_win = multi_in & (u2_mod == max_signal) & (u1_mod < max_signal)
        p3_win = multi_in & (u3_mod == max_signal) & (u1_mod < max_signal) & (u2_mod < max_signal)
        
        # Number of other players "in" for each multi-in trial: in_count - 1.
        num_others = in_count[multi_in] - 1
        
        # Winner gets payoff = 1 + (number of other players in)
        payoff1[p1_win] = 1 + num_others[np.where(p1_win[multi_in])[0]]
        payoff2[p2_win] = 1 + num_others[np.where(p2_win[multi_in])[0]]
        payoff3[p3_win] = 1 + num_others[np.where(p3_win[multi_in])[0]]
        
        # Losers who are in get -1.
        cond = multi_in & p1_in & (~p1_win)
        payoff1[cond] = -1
        cond = multi_in & p2_in & (~p2_win)
        payoff2[cond] = -1
        cond = multi_in & p3_in & (~p3_win)
        payoff3[cond] = -1
    
    # If no one is in (in_count==0), payoffs remain 0.
    
    avg_payoff1 = np.mean(payoff1)
    avg_payoff2 = np.mean(payoff2)
    avg_payoff3 = np.mean(payoff3)
    
    return avg_payoff1, avg_payoff2, avg_payoff3

In [3]:
def compute_finite_diff_gradient(state, num_trials, h=0.01):
    """
    For each cutoff parameter in the state vector, compute the finite difference derivative
    of the corresponding player's average payoff with respect to that cutoff.
    
    state: [c1, c2_in, c2_out, c3_in_in, c3_in_out, c3_out_in]
      - For state[0]: use Player 1's payoff.
      - For state[1] and state[2]: use Player 2's payoff.
      - For state[3], state[4], state[5]: use Player 3's payoff.
      
    Returns:
        grad: numpy array of 6 finite-difference derivative estimates.
    """
    state = np.array(state)
    grad = np.zeros_like(state)
    for i in range(len(state)):
        state_plus = state.copy()
        state_minus = state.copy()
        state_plus[i] = np.clip(state_plus[i] + h, 0, 1)
        state_minus[i] = np.clip(state_minus[i] - h, 0, 1)
        
        payoffs_plus = simulate_game_3players(*state_plus, num_trials=num_trials)
        payoffs_minus = simulate_game_3players(*state_minus, num_trials=num_trials)
        
        if i == 0:  # Player 1's cutoff
            f_plus = payoffs_plus[0]
            f_minus = payoffs_minus[0]
        elif i in [1, 2]:  # Player 2's cutoffs
            f_plus = payoffs_plus[1]
            f_minus = payoffs_minus[1]
        elif i in [3, 4, 5]:  # Player 3's cutoffs
            f_plus = payoffs_plus[2]
            f_minus = payoffs_minus[2]
        
        # Finite difference derivative estimate.
        grad[i] = (f_plus - f_minus) / (2 * h)
    return grad

In [18]:
def gradient_ascent(state, num_trials, lr, max_iters=1e6, h=0.01, tol=1e-4):
    """
    Iteratively update the cutoff parameters using a gradient-ascent scheme.
    For each parameter, we update:
    
        new_state[i] = state[i] + lr * (finite difference derivative for that parameter)
    
    This moves the cutoff in the direction that increases the corresponding player's payoff.
    We assume that at equilibrium the derivative should be zero.
    
    Parameters:
        state: initial 6D vector of cutoff parameters.
        num_trials: number of simulation trials for finite-difference estimation.
        iterations: number of gradient ascent steps.
        lr: learning rate.
        h: finite difference step size.
        
    Returns:
        state: final state after gradient ascent.
    """
    state = np.array(state)
    for iteration in range(1,max_iters):
        grad = compute_finite_diff_gradient(state, num_trials, h)
        # Update each cutoff in the direction of its derivative.
        # (If a derivative is very negative, this update will lower the cutoff.)
        state = state + lr/np.sqrt(iteration) * grad
        # Clip state to remain in [0, 1].
        state = np.clip(state, 0, 1)
        if iteration % 100 == 0:
            payoffs = simulate_game_3players(*state, num_trials=num_trials)
            print(f"Iteration {iteration:4d} | State: {state} | Payoffs: {payoffs} | Gradients: {grad}")
        if np.linalg.norm(grad)<tol:
            print(f"converged on iteration {iteration}")
            return state
    print("did not converge")
    return state

In [19]:
num_trials = 100_000
max_iterations = 1000
lr = 0.1
h = 0.001

# Initial guess for the six cutoff values.
# [c1, c2_in, c2_out, c3_in_in, c3_in_out, c3_out_in]
initial_state = [0.5, 0.8, 0.25, 0.9, 0.6, 0.5]

final_state = gradient_ascent(initial_state, num_trials, lr, max_iters=max_iterations, h=h)

final_payoffs = simulate_game_3players(*final_state, num_trials=num_trials)
print("\nFinal state found (via gradient ascent):")
print(f"  c1        = {final_state[0]:.3f}")
print(f"  c2_in     = {final_state[1]:.3f}")
print(f"  c2_out    = {final_state[2]:.3f}")
print(f"  c3_in_in  = {final_state[3]:.3f}")
print(f"  c3_in_out = {final_state[4]:.3f}")
print(f"  c3_out_in = {final_state[5]:.3f}")
print(f"Final average payoffs: {final_payoffs}")

Iteration  100 | State: [0.51756103 0.72281847 0.24768287 0.85751273 0.66579439 0.49496998] | Payoffs: (0.21197, 0.3231, 0.46493) | Gradients: [ 0.     0.     0.     0.    -0.03   0.055]
Iteration  200 | State: [0.51756103 0.72281847 0.24768287 0.85751273 0.66599532 0.49474908] | Payoffs: (0.21193, 0.3231, 0.46497) | Gradients: [0.    0.    0.    0.    0.    0.025]
converged on iteration 221

Final state found (via gradient ascent):
  c1        = 0.518
  c2_in     = 0.723
  c2_out    = 0.248
  c3_in_in  = 0.858
  c3_in_out = 0.666
  c3_out_in = 0.495
Final average payoffs: (0.21193, 0.32309, 0.46498)
